In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import csv
from sklearn.model_selection import GridSearchCV

## Lần 1 với tham số custom từ bên votting

In [5]:
file_paths_basic = [
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_features_v2.csv'
]

params = {
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv': {
        'knn': {'n_neighbors': 4, 'metric': 'euclidean'},
        'ann': {'activation':'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (150,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 100, 'gamma':0.01},
        'rf': {'n_estimators': 1200, 'max_features': 6, 'criterion':'entropy', 'max_depth': 10}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv': {
        'knn': {'n_neighbors': 12, 'metric': 'euclidean','weights': 'distance'},
        'ann': {'alpha': 0.05, 'hidden_layer_sizes': (50,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 100, 'gamma':0.01},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 10}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv': {
        'knn': {'n_neighbors': 4, 'metric': 'manhattan'},
        'ann': {'activation':'tanh', 'hidden_layer_sizes': (50,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 100, 'gamma':0.01},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 6,'n_estimators':500}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv': {
        'knn': {'n_neighbors': 4, 'metric': 'euclidean'},
        'ann': {'activation':'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (150,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 10, 'gamma':0.01},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 10,'n_estimators':200}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv': {
        'knn': {'n_neighbors': 10, 'metric': 'euclidean'},
        'ann': {'hidden_layer_sizes': (50,), 'alpha': 0.05, 'max_iter': 1000},
        'svm': {'kernel': 'linear', 'C': 10, 'gamma':1},
        'rf': {'max_features': 6, 'max_depth': 10}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_features_v2.csv': {
        'knn': {'n_neighbors': 6, 'metric': 'euclidean','weights': 'distance'},
        'ann': {'activation':'tanh', 'hidden_layer_sizes': (150,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 100, 'gamma':0.01},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 6,'n_estimators':500}
    }
}

results = []
accuracies = []

for file_path in file_paths_basic:
    data = pd.read_csv(file_path)
    X = data.drop(['filename', 'label'], axis=1)
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    # Fetch the parameters for the current file
    param_set = params[file_path]
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(**param_set['knn'])
    ann = MLPClassifier(**param_set['ann'], random_state=42)
    svm = SVC(**param_set['svm'], random_state=42)
    rf = RandomForestClassifier(**param_set['rf'], random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, penalty='l1', solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%'
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'C:\AIL301m\result\Stacking\Basic_results_4_models_v3.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

File: C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 94.65%
Precision: 94.67%
Recall: 94.65%
F-measure: 94.64%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.76%
Precision: 98.76%
Recall: 98.76%
F-measure: 98.76%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.10%
Precision: 98.12%
Recall: 98.10%
F-measure: 98.10%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.09%
Precision: 98.10%
Recall: 98.09%
F-measure: 98.09%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.34%
Precision: 98.34%
Recall: 98.34%
F-measure: 98.34%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_feature

## Lần 2 với tham số từ Stacking

In [7]:
file_paths_basic = [
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_features_v2.csv'
]

params = {
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv': {
        'knn': {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'uniform'},
        'ann': {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (150,), 'max_iter': 1000},
        'svm': {'C': 100, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'},
        'rf': {'criterion': 'entropy', 'max_depth': 20, 'max_features': 6, 'n_estimators': 1500}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv': {
        'knn': {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'distance'},
        'ann': {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (150,), 'max_iter': 1000},
        'svm': {'C': 10, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 10}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv': {
        'knn': {'metric': 'manhattan', 'n_neighbors': 10, 'weights': 'distance'},
        'ann': {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (150,), 'max_iter': 1000},
        'svm': {'C': 1000, 'degree': 3, 'gamma': 0.01, 'kernel': 'linear'},
        'rf': {'criterion': 'entropy', 'max_depth': 20, 'max_features': 3, 'n_estimators': 1500}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv': {
        'knn': {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'uniform'},
        'ann': {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (100,), 'max_iter': 1000},
        'svm': {'C': 100, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'},
        'rf': {'criterion': 'entropy', 'max_depth': 10, 'max_features': 6, 'n_estimators': 1500}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv': {
        'knn': {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'uniform'},
        'ann': {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (150,), 'max_iter': 1000},
        'svm': {'C': 1000, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'},
        'rf': {'criterion': 'entropy', 'max_depth': 6, 'max_features': 10, 'n_estimators': 100}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_features_v2.csv': {
        'knn': {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'uniform'},
        'ann': {'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': (100,), 'max_iter': 1000},
        'svm': {'C': 10, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'},
        'rf': {'criterion': 'entropy', 'max_depth': 20, 'max_features': 6, 'n_estimators': 1200}
    }
}

results = []
accuracies = []

for file_path in file_paths_basic:
    data = pd.read_csv(file_path)
    X = data.drop(['filename', 'label'], axis=1)
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    # Fetch the parameters for the current file
    param_set = params[file_path]
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(**param_set['knn'])
    ann = MLPClassifier(**param_set['ann'], random_state=42)
    svm = SVC(**param_set['svm'], random_state=42)
    rf = RandomForestClassifier(**param_set['rf'], random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, penalty='l1', solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%'
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'C:\AIL301m\result\Stacking\Basic_results_4_models_v3_stack.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

File: C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 94.65%
Precision: 94.65%
Recall: 94.65%
F-measure: 94.64%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.25%
Precision: 98.25%
Recall: 98.25%
F-measure: 98.25%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.00%
Precision: 98.01%
Recall: 98.00%
F-measure: 98.00%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 97.99%
Precision: 98.01%
Recall: 97.99%
F-measure: 97.99%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.34%
Precision: 98.34%
Recall: 98.34%
F-measure: 98.34%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_feature

## Maxxing

In [2]:
file_paths_basic = [
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv',
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_features_v2.csv'
]

params = {
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv': {
        'knn': {'n_neighbors': 4, 'metric': 'euclidean'},
        'ann': {'activation':'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (150,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 100, 'gamma':0.01},
        'rf': {'n_estimators': 1200, 'max_features': 6, 'criterion':'entropy', 'max_depth': 10}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv': {
        'knn': {'n_neighbors': 12, 'metric': 'euclidean','weights': 'distance'},
        'ann': {'alpha': 0.05, 'hidden_layer_sizes': (50,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 100, 'gamma':0.01},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 10}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv': {
        'knn': {'n_neighbors': 4, 'metric': 'manhattan'},
        'ann': {'activation':'tanh', 'hidden_layer_sizes': (50,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 100, 'gamma':0.01},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 6,'n_estimators':500}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv': {
        'knn': {'n_neighbors': 4, 'metric': 'euclidean'},
        'ann': {'activation':'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (150,),'max_iter': 1000},
        'svm': {'kernel': 'rbf', 'C': 10, 'gamma':0.01},
        'rf': {'max_features': 6, 'criterion':'entropy', 'max_depth': 10,'n_estimators':200}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv': {
        'knn': {'n_neighbors': 10, 'metric': 'euclidean'},
        'ann': {'hidden_layer_sizes': (50,), 'alpha': 0.05, 'max_iter': 1000},
        'svm': {'kernel': 'linear', 'C': 10, 'gamma':1},
        'rf': {'max_features': 6, 'max_depth': 10}
    },
    r'C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_features_v2.csv': {
        'knn': {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'uniform'},
        'ann': {'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': (100,), 'max_iter': 1000},
        'svm': {'C': 10, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'},
        'rf': {'criterion': 'entropy', 'max_depth': 20, 'max_features': 6, 'n_estimators': 1200}
    }
}

results = []
accuracies = []

for file_path in file_paths_basic:
    data = pd.read_csv(file_path)
    X = data.drop(['filename', 'label'], axis=1)
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    # Fetch the parameters for the current file
    param_set = params[file_path]
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(**param_set['knn'])
    ann = MLPClassifier(**param_set['ann'], random_state=42)
    svm = SVC(**param_set['svm'], random_state=42)
    rf = RandomForestClassifier(**param_set['rf'], random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%'
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'C:\AIL301m\result\Stacking\Finale.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

File: C:\AIL301m\Feature extract\basic\basicv2\Merged_BC-15_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 94.65%
Precision: 94.67%
Recall: 94.65%
F-measure: 94.64%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Huong_thom-1_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.69%
Precision: 98.69%
Recall: 98.69%
F-measure: 98.69%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Nep-87_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.10%
Precision: 98.12%
Recall: 98.10%
F-measure: 98.10%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Q-5_modify_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.09%
Precision: 98.10%
Recall: 98.09%
F-measure: 98.09%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Thien_uu-8_features_v2.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 98.34%
Precision: 98.34%
Recall: 98.34%
F-measure: 98.34%
File: C:\AIL301m\Feature extract\basic\basicv2\Merged_Xi-23_feature